### **Bronze layer**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

review_schema = StructType([
    StructField("review_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("business_id", StringType(), True),
    StructField("stars", DoubleType(), True),
    StructField("useful", IntegerType(), True),
    StructField("funny", IntegerType(), True),
    StructField("cool", IntegerType(), True),
    StructField("text", StringType(), True),
    StructField("date", StringType(), True) 
])

review_bronze = (
    spark.read
    .schema(review_schema)
    .json("/Volumes/workspace/default/yelp-reviews/review_*.json")
    .withColumn("_ingest_ts", F.current_timestamp())  
)

review_bronze.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("bronzereview")


### **Silver Layer**

In [0]:
from pyspark.sql import *

src = spark.table("bronzereview")
print("Row count:", src.count())

In [0]:
key_cols = ["review_id", "user_id", "business_id", "text", "stars", "date"]
missing_stats = src.select(
    *[F.sum(F.col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in key_cols]
)
missing_stats.display()

In [0]:
dupes_by_id = (
    src.groupBy("review_id")
       .count()
       .filter("count > 1")
       .orderBy(F.col("count").desc())
)
print("Duplicate review_id rows:", dupes_by_id.count())

dupes_composite = (
    src.groupBy("user_id", "business_id", "date", "text")
       .count()
       .filter("count > 1")
       .orderBy(F.col("count").desc())
)
print("Potential duplicate (user,business,date,text):", dupes_composite.count())
dupes_composite.display()

In [0]:
src = src.dropDuplicates(["user_id", "business_id", "date", "text"])
print("Row count:", src.count())

In [0]:
stars_invalid = src.filter((F.col("stars") < 1) | (F.col("stars") > 5))
print("Stars out of [1,5]:", stars_invalid.count())

useful_neg = src.filter(F.col("useful") < 0)
funny_neg  = src.filter(F.col("funny")  < 0)
cool_neg   = src.filter(F.col("cool")   < 0)
print("Negative useful:", useful_neg.count(), "Negative funny:", funny_neg.count(), "Negative cool:", cool_neg.count())

useful_neg.display()
funny_neg.display()
cool_neg.display()

In [0]:
src = (
    src
    .withColumn("useful", F.abs(F.col("useful")))
    .withColumn("funny",  F.abs(F.col("funny")))
    .withColumn("cool",   F.abs(F.col("cool")))
)

print("Before:", src.count(), "After:", src.count())
src.select(
    F.min("useful"), F.min("funny"), F.min("cool")
).show()

In [0]:
src_with_date = src.withColumn("date_parsed", F.to_date("date"))
by_year_month = (
    src_with_date
    .groupBy(F.year("date_parsed").alias("year"), F.month("date_parsed").alias("month"))
    .count()
    .orderBy("year","month")
)
by_year_month.display()

In [0]:
stats = (
    src  # your reviews DataFrame/table
    .withColumn("text_trim", F.trim("text"))
    .withColumn("text_len", F.length("text_trim"))
    .select(
        F.min("text_len").alias("min_text_len"),
        F.max("text_len").alias("max_text_len"),
        F.avg("text_len").alias("avg_text_len")
    )
)

stats.show(truncate=False)

In [0]:
from pyspark.sql.functions import col, split

review_silver = src.filter(col("review_id").isNotNull())

review_silver.write.format("delta").mode("overwrite") \
.saveAsTable("silverreview")